In [1]:
!export AWS_S3_ENDPOINT_URL=http://localhost:8000
!export AWS_ACCESS_KEY_ID=DevAccessKey
!export AWS_SECRET_ACCESS_KEY=DevSecretKey

In [2]:
import wandb
wandb.login(key='local-8c4310dcf3cb371cac7343a466e09de6b04e4282', host='http://localhost:8080')
ENTITY = None

In [3]:
import ast
import sys
import json
import boto3
from pathlib import Path
from dill.source import getsource
from dill import detect

import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt

from scipy.stats import ks_2samp
from sklearn import metrics
from sklearn import model_selection
import xgboost as xgb

pd.set_option('display.max_columns', None)

In [4]:
data_dir = Path('../data/')
model_dir = Path('../models')
model_dir.mkdir(exist_ok=True)

id_vars = ['UniqueID']
targ_var = 'loan_default'

In [5]:
def function_to_string(fn):
    return getsource(detect.code(fn))

In [6]:
run = wandb.init(entity=ENTITY, name="score1", project='credit_scorecard', job_type='preprocess-data', config={'wandb_nb':'wandb_credit_soc'})  # config is optional here

In [7]:
from src.data_utils import (
    describe_data_g_targ,
    one_hot_encode_data,
    create_feature_interaction_constraints,
    get_monotonic_constraints,
    load_training_data,
    calculate_credit_scores
)

from src.scorecard import generate_scorecard

In [8]:
dataset = pd.read_csv(data_dir/'vehicle_loans_subset.csv')
dataset.head()

   UniqueID    LTV  manufacturer_id Employment_Type  State_ID  \
0    417428  88.17               86   Self employed         4   
1    417429  83.78               86        Salaried         8   
2    417430  77.39               86   Self employed         6   
3    417431  72.14               86   Self employed         3   
4    417432  85.92               86   Self employed         6   

   PERFORM_CNS_SCORE  AgeInMonths  DaysSinceDisbursement  loan_default  
0              681.0        638.0                  153.0             0  
1              384.0        478.0                  153.0             0  
2                NaN        441.0                  153.0             0  
3                NaN        384.0                  153.0             0  
4              721.0        343.0                  153.0             0  

In [9]:
dataset, p_vars = one_hot_encode_data(dataset, id_vars, targ_var)

In [10]:
processed_data_path = data_dir/'proc_ds.csv'
dataset.to_csv(processed_data_path, index=False)

In [11]:
processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/process')

In [12]:
import ast
import sys
import os
import json
import boto3
from pathlib import Path
from dill.source import getsource
from dill import detect

import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt

from scipy.stats import ks_2samp
from sklearn import metrics
from sklearn import model_selection
import xgboost as xgb

pd.set_option('display.max_columns', None)

In [13]:
import ast
import sys
import os
import json
import boto3
from pathlib import Path
from dill.source import getsource
from dill import detect

import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt

from scipy.stats import ks_2samp
from sklearn import metrics
from sklearn import model_selection
import xgboost as xgb

pd.set_option('display.max_columns', None)

In [14]:
os.environ["AWS_S3_ENDPOINT_URL"] = "http://localhost:8000"
os.environ["AWS_ACCESS_KEY_ID"] = "DevAccessKey"
os.environ["AWS_SECRET_ACCESS_KEY"] = "DevSecretKey"

In [15]:
processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/process')

[]

In [16]:
# Attach our processed data to the Artifact
processed_ds_art.add_file(processed_data_path)

<ManifestEntry digest: cslJubeoq5UrWhqw3sBeqw==>

In [17]:
run.log_artifact(processed_ds_art)
run.finish()

In [18]:
processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

[]

In [19]:
# Attach our processed data to the Artifact
processed_ds_art.add_file(processed_data_path)

<ManifestEntry digest: cslJubeoq5UrWhqw3sBeqw==>

In [20]:
run.log_artifact(processed_ds_art)
run.finish()

In [21]:
run = wandb.init(entity=ENTITY, name="score1", project='credit_scorecard', job_type='preprocess-data', config={'wandb_nb':'wandb_credit_soc'})  # config is optional here

In [22]:
from src.data_utils import (
    describe_data_g_targ,
    one_hot_encode_data,
    create_feature_interaction_constraints,
    get_monotonic_constraints,
    load_training_data,
    calculate_credit_scores
)

from src.scorecard import generate_scorecard

In [23]:
dataset = pd.read_csv(data_dir/'vehicle_loans_subset.csv')
dataset.head()

   UniqueID    LTV  manufacturer_id Employment_Type  State_ID  \
0    417428  88.17               86   Self employed         4   
1    417429  83.78               86        Salaried         8   
2    417430  77.39               86   Self employed         6   
3    417431  72.14               86   Self employed         3   
4    417432  85.92               86   Self employed         6   

   PERFORM_CNS_SCORE  AgeInMonths  DaysSinceDisbursement  loan_default  
0              681.0        638.0                  153.0             0  
1              384.0        478.0                  153.0             0  
2                NaN        441.0                  153.0             0  
3                NaN        384.0                  153.0             0  
4              721.0        343.0                  153.0             0  

In [24]:
dataset, p_vars = one_hot_encode_data(dataset, id_vars, targ_var)

In [25]:
processed_data_path = data_dir/'proc_ds.csv'
dataset.to_csv(processed_data_path, index=False)

In [26]:
processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

[]

In [27]:
# Attach our processed data to the Artifact
processed_ds_art.add_file(processed_data_path)

<ManifestEntry digest: cslJubeoq5UrWhqw3sBeqw==>

In [28]:
run.log_artifact(processed_ds_art)
run.finish()

In [29]:
run = wandb.init(entity=ENTITY, name="score2", project='credit_scorecard', job_type='preprocess-data', config={'wandb_nb':'wandb_credit_soc'})  # config is optional here

In [30]:
from src.data_utils import (
    describe_data_g_targ,
    one_hot_encode_data,
    create_feature_interaction_constraints,
    get_monotonic_constraints,
    load_training_data,
    calculate_credit_scores
)

from src.scorecard import generate_scorecard

In [31]:
dataset = pd.read_csv(data_dir/'vehicle_loans_subset.csv')
dataset.head()

   UniqueID    LTV  manufacturer_id Employment_Type  State_ID  \
0    417428  88.17               86   Self employed         4   
1    417429  83.78               86        Salaried         8   
2    417430  77.39               86   Self employed         6   
3    417431  72.14               86   Self employed         3   
4    417432  85.92               86   Self employed         6   

   PERFORM_CNS_SCORE  AgeInMonths  DaysSinceDisbursement  loan_default  
0              681.0        638.0                  153.0             0  
1              384.0        478.0                  153.0             0  
2                NaN        441.0                  153.0             0  
3                NaN        384.0                  153.0             0  
4              721.0        343.0                  153.0             0  

In [32]:
dataset, p_vars = one_hot_encode_data(dataset, id_vars, targ_var)

In [33]:
processed_data_path = data_dir/'proc_ds.csv'
dataset.to_csv(processed_data_path, index=False)

In [34]:
s3_client = boto3.client('s3')
s3_client.upload_file('proc_ds.csv', 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [35]:
s3_client = boto3.client('s3')
s3_client.upload_file(data_dir/'proc_ds.csv', 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [36]:
s3_client = boto3.client('s3')
s3_client.upload_file(data_dir/'proc_ds.csv', 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [37]:
s3_client = boto3.client('s3')

In [38]:
s3_client.upload_file(data_dir/'proc_ds.csv', 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [39]:
s3_client.upload_file(data_dir + '/proc_ds.csv', 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [40]:
s3_client.upload_file((data_dir/'proc_ds.csv'.__str__()), 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [41]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__()), 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [42]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__(), 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [43]:
os.environ["AWS_S3_ENDPOINT_URL"] = "http://localhost:8000"
os.environ["AWS_ACCESS_KEY_ID"] = "DevAccessKey"
os.environ["AWS_SECRET_ACCESS_KEY"] = "DevSecretKey"

In [44]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__(), 'creditscore', 'datasets/vehicle_defaults_processed')

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [45]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__(), 'creditscore', 'datasets/vehicle_defaults_processed')

In [46]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__(), 'creditscore', 'datasets/vehicle_defaults_processed')

In [47]:
os.environ["AWS_S3_ENDPOINT_URL"] = "http://localhost:8000"
os.environ["AWS_ACCESS_KEY_ID"] = "DevAccessKeym"
os.environ["AWS_SECRET_ACCESS_KEY"] = "DevSecretKey"

In [48]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__(), 'creditscore', 'datasets/vehicle_defaults_processed')

In [49]:
s3_client = boto3.client('s3', endpoint_url="http://localhost:8000",
                         aws_access_key_id="DevAccessKey", aws_secret_access_key="DevSecretKey")

In [50]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__(), 'creditscore', 'datasets/vehicle_defaults_processed')

In [51]:

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [52]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__(), 'creditscore', 'datasets/vehicle_defaults_processed1')

In [53]:

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

In [54]:
os.environ["AWS_S3_ENDPOINT_URL"] = "http://localhost:8000"
os.environ["AWS_ACCESS_KEY_ID"] = "DevAccessKey"
os.environ["AWS_SECRET_ACCESS_KEY"] = "DevSecretKey"

In [55]:

processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')

[<ManifestEntry ref: s3://creditscore/datasets/vehicle_defaults_processed/vehicle_defaults_processed>]

In [56]:
# Attach our processed data to the Artifact
processed_ds_art.add_file(processed_data_path)

<ManifestEntry digest: cslJubeoq5UrWhqw3sBeqw==>

In [57]:
run.log_artifact(processed_ds_art)
run.finish()

In [58]:
with wandb.init(entity=ENTITY, name="model1", project='credit_scorecard', job_type='train-val-split', config={'wandb_nb':'wandb_credit_soc'}) as run:     # config is optional here

    # Download the subset of the vehicle loan default data from W&B
    dataset_art = run.use_artifact('vehicle_defaults_processed:latest', type='processed_dataset')
    dataset_dir = dataset_art.download(data_dir)
    dataset = pd.read_csv(processed_data_path)

    # Set Split Params
    test_size = 0.25
    random_state = 42

    # Log the splilt params
    run.config.update({'test_size':test_size, 'random_state': random_state})

    # Do the Train/Val Split
    trndat, valdat = model_selection.train_test_split(dataset, test_size=test_size,
                                                      random_state=random_state, stratify=dataset[[targ_var]])

    print(f'Train dataset size: {trndat[targ_var].value_counts()} \n')
    print(f'Validation dataset sizeL {valdat[targ_var].value_counts()}')

    # Save split datasets
    train_path = data_dir/'train.csv'
    val_path = data_dir/'val.csv'
    trndat.to_csv(train_path, index=False)
    valdat.to_csv(val_path, index=False)


    # Create a new artifact for the processed data, including the function that created it, to Artifacts
    split_ds_art = wandb.Artifact(name='vehicle_defaults_split',
                                  type='train-val-dataset',
                                  description='Processed dataset split into train and valiation',
                                  metadata={'test_size': test_size, 'random_state': random_state}
                                  )
    split_ds_art.add_reference('s3://creditscore/artefacts')

    # Attach our processed data to the Artifact
    split_ds_art.add_file(train_path)
    split_ds_art.add_file(val_path)

    # Log the Artifact
    run.log_artifact(split_ds_art)

In [59]:
with wandb.init(entity=ENTITY, name="split2", project='credit_scorecard', job_type='train-val-split', config={'wandb_nb':'wandb_credit_soc'}) as run:     # config is optional here

    # Download the subset of the vehicle loan default data from W&B
    dataset_art = run.use_artifact('vehicle_defaults_processed:latest', type='processed_dataset')
    dataset_dir = dataset_art.download(data_dir)
    dataset = pd.read_csv(processed_data_path)

    # Set Split Params
    test_size = 0.25
    random_state = 42

    # Log the splilt params
    run.config.update({'test_size':test_size, 'random_state': random_state})

    # Do the Train/Val Split
    trndat, valdat = model_selection.train_test_split(dataset, test_size=test_size,
                                                      random_state=random_state, stratify=dataset[[targ_var]])

    print(f'Train dataset size: {trndat[targ_var].value_counts()} \n')
    print(f'Validation dataset sizeL {valdat[targ_var].value_counts()}')

    # Save split datasets
    train_path = data_dir/'train.csv'
    val_path = data_dir/'val.csv'
    trndat.to_csv(train_path, index=False)
    valdat.to_csv(val_path, index=False)

    s3_client.upload_file(train_path.__str__(), 'creditscore', 'artifacts/train.csv')
    s3_client.upload_file(val_path.__str__(), 'creditscore', 'artifacts/val.csv')

    # Create a new artifact for the processed data, including the function that created it, to Artifacts
    split_ds_art = wandb.Artifact(name='vehicle_defaults_split',
                                  type='train-val-dataset',
                                  description='Processed dataset split into train and valiation',
                                  metadata={'test_size': test_size, 'random_state': random_state}
                                  )
    split_ds_art.add_reference('s3://creditscore/artifacts')

    # Attach our processed data to the Artifact
    # split_ds_art.add_file('train.csv')
    # split_ds_art.add_file('val.csv')

    # Log the Artifact
    run.log_artifact(split_ds_art)

In [60]:
with wandb.init(entity=ENTITY, name="split3", project='credit_scorecard', job_type='train-val-split', config={'wandb_nb':'wandb_credit_soc'}) as run:     # config is optional here

    # Download the subset of the vehicle loan default data from W&B
    dataset_art = run.use_artifact('vehicle_defaults_processed:latest', type='processed_dataset')
    dataset_dir = dataset_art.download(data_dir)
    dataset = pd.read_csv(processed_data_path)

    # Set Split Params
    test_size = 0.25
    random_state = 42

    # Log the splilt params
    run.config.update({'test_size':test_size, 'random_state': random_state})

    # Do the Train/Val Split
    trndat, valdat = model_selection.train_test_split(dataset, test_size=test_size,
                                                      random_state=random_state, stratify=dataset[[targ_var]])

    print(f'Train dataset size: {trndat[targ_var].value_counts()} \n')
    print(f'Validation dataset sizeL {valdat[targ_var].value_counts()}')

    # Save split datasets
    train_path = data_dir/'train.csv'
    val_path = data_dir/'val.csv'
    trndat.to_csv(train_path, index=False)
    valdat.to_csv(val_path, index=False)

    s3_client.upload_file(train_path.__str__(), 'creditscore', 'artifacts/train.csv')
    s3_client.upload_file(val_path.__str__(), 'creditscore', 'artifacts/val.csv')

    # Create a new artifact for the processed data, including the function that created it, to Artifacts
    split_ds_art = wandb.Artifact(name='vehicle_defaults_split',
                                  type='train-val-dataset',
                                  description='Processed dataset split into train and valiation',
                                  metadata={'test_size': test_size, 'random_state': random_state}
                                  )
    split_ds_art.add_reference('http://localhost:8001/creditscore/artifacts')

    # Attach our processed data to the Artifact
    # split_ds_art.add_file('train.csv')
    # split_ds_art.add_file('val.csv')

    # Log the Artifact
    run.log_artifact(split_ds_art)